# Assessent of mitosis-mediated phototoxicity temporal footprint

The data is assumed to be organised in a folder as follows:

   ```
   Raw-images (folder)
   |
   |-Biological-replica-date-1 (folder)
       |
       |-- control-condition (folder)
       |    |  file1.tif
       |    |  file2.tif
       |    |  ...
       |
       |-- condition1 (folder)
       |    |  file1.tif
       |    |  file2.tif
       |    |  ...
       |
       |-- condition2 (folder)
       |    |  file1.tif
       |    |  file2.tif
       |    |  ...
    Masks (folder)
    |
    |--Biological-replica-date-1 (folder)
           |
           |-- control-condition (folder)
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition1 (folder)
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition2 (folder)
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
   ```